# Hazard assessment for Infrastructures using Euro-Cordex datasets
## Calculation of the indicator "Number of Days Abover 35°C, 40°C, and 45°C"

- See our [how to use risk workflows](https://handbook.climaax.eu/notebooks/workflows_how_to.html) page for information on how to run this notebook.

## Hazard assessment methodology
We utilized outputs from 14 models within the EURO-CORDEX framework to evaluate hazards affecting infrastructure, in this notebook we used the daily maximum temperature as an indicator of the hazard. Our analysis included three Representative Concentration Pathways (RCPs): RCP2.6, RCP4.5, and RCP8.5. To structure the future projections, we divided the RCP scenarios into three distinct periods: 2021–2050, 2041–2070, and 2071–2100. Additionally, we used the historical period (1981–2010) as a baseline for comparison.

For each period and model, the variation of the indicator "number of days above 35°C, 40°C, and 45°C" is calculated by comparing the future scenarios with the historical data. Finally, once the results for each model are obtained, the ensemble mean is computed. The results of these computations will be visualized in the notebook *04_cordex_tempDaysAbove.ipynb* to assess potential future hazards.

## Limitation of the Euro-Cordex dataset
The EURO-CORDEX (Coordinated Regional Climate Downscaling Experiment for Europe) project is a set of high-resolution regional climate projections for Europe, designed to support impact, adaptation, and vulnerability assessments under various climate change scenarios. The EURO-CORDEX integrate global climate model (GCM) outputs with regional climate models (RCMs), enabling the simulation of climatic patterns and extremes.
The models explore different Representative Concentration Pathways (RCPs) from CMIP5 (RCP2.6, RCP4.5, RCP8.5) and Shared Socioeconomic Pathways (SSPs) from CMIP6 (SSP1-2.6, SSP5-8.5). The simulations cover historical periods (1950–2005) and future projections (2006–2100). These models are validated against observational data and reanalysis datasets.

Some of the limitations:
- EURO-CORDEX offers high-resolution data (typically 0.11° ~ 12.5 km and 0.44° ~ 50 km), it may still not fully capture localized phenomena such as urban heat islands, small-scale topographic effects, and small meteorological events.
- Like all climate models, EURO-CORDEX RCMs and their driving GCMs exhibit biases compared to observed data, these Biases can vary regionally and seasonally. And may struggle to accurately simulate extreme weather events such as heatwaves, heavy precipitation, or storms.
- While the dataset captures trends in extremes, very high thresholds (>45°C or >100 mm/day rainfall) may have higher uncertainty due to limited observational data.

## Preparation work

All the EURO-CORDEX models used in this workflow are freely available on copernicus C3S platform (https://cds.climate.copernicus.eu/datasets/projections-cordex-domains-single-levels?tab=overview), and Downloaded through the implemented API, the data were then processed to ensure that the grid type was consistent across all models and to fill any gaps in the dates. Here an example for one model

In [ ]:
import cdsapi

dataset = "projections-cordex-domains-single-levels"
request = {
    "domain": "europe",
    "experiment": "historical",
    "horizontal_resolution": "0_11_degree_x_0_11_degree",
    "temporal_resolution": "daily_mean",
    "variable": ["2m_air_temperature"],
    "gcm_model": "ncc_noresm1_m",
    "rcm_model": "mohc_hadrem3_ga7_05",
    "ensemble_member": "r1i1p1",
    "start_year": ["1981"],
    "end_year": ["1985"]
}

client = cdsapi.Client()
client.retrieve(dataset, request).download()


### Select area of interest
Before downloading the data, we will define the coordinates of the area of interest, for this workflow we selected the Italy region. Based on the shapefile of the country we will be able to clip the datasets for further processing, and display hazard and damage maps for this area.

### Load libraries

In [ ]:
from collections import defaultdict
import os
import xarray as xr
import xclim.indices
import re

### Create the directory structure

In [ ]:

nc_files = "/work/cmcc/dg07124/climax/data/cordex/temp"
general_path = "/work/cmcc/dg07124/climax/indicators/cordex/temp"
subfolders = ['historical','rcp26', 'rcp45', 'rcp85']

In [ ]:
# Temperature thresholds
thresholds = ['35 C', '40 C', '45 C']

# Time ranges to process
rcp_time_ranges = [('2021', '2050'), ('2041', '2070'), ('2071', '2100')]
historical_time_range = [('1981', '2010')]

In [ ]:
# Function to process each NetCDF file for a given time range
def process_file(file_path, threshold, save_path, start_year, end_year):
    print("---------------------------------------------------")
    print(f"Processing {file_path} for time range {start_year}-{end_year}")
    ds = xr.open_dataset(file_path)

    # Select daily max temperature for the given time range
    ds_sliced = ds.sel(time=slice(start_year, end_year))
    # Convert to Celcius
    dailyMaxTemp = (ds_sliced['tasmax'] - 273.15).resample(time='D').max()
    dailyMaxTemp.attrs['units'] = 'C'

    # Get the minimum and maximum values
    min_value = dailyMaxTemp.min(skipna=True).item()  # Convert to a scalar with .item()
    max_value = dailyMaxTemp.max(skipna=True).item()

    print(f"Temp min value: {min_value}")
    print(f"Temp max value: {max_value}")


    # Calculate the number of days above the threshold using xclim
    with xclim.set_options(cf_compliance="log"):
        #NumbDaysAbove = xclim.atmos.tx_days_above(tasmax=dailyMaxTemp, thresh=threshold, freq="YS")
        NumbDaysAbove = xclim.indices.tx_days_above(dailyMaxTemp, thresh=threshold, freq='YS', op='>')

    # Average over the time dimension (years in this case)
    NumbDaysAbove_avg = NumbDaysAbove.mean(dim='time', skipna=True)

    # Create the new filename with the time range and threshold information
    filename = os.path.basename(file_path)  # Extract original filename
    file_name_no_ext = os.path.splitext(filename)[0]  # Remove extension
    number_threshold = re.findall(r'\d+', threshold)[0]  # get only the number of the threshold
    new_filename = f"{file_name_no_ext}_above{number_threshold}_{start_year}-{end_year}.nc"

    # Save the result to the new file path
    NumbDaysAbove_avg.to_netcdf(os.path.join(save_path, new_filename))

    # Get the minimum and maximum values
    min_value_indic = NumbDaysAbove_avg.min(skipna=True).item()  # Convert to a scalar with .item()
    max_value_indic = NumbDaysAbove_avg.max(skipna=True).item()

    # Print the results
    print(f"Minimum value days above: {min_value_indic}")
    print(f"Maximum value days above: {max_value_indic}")

    print(f"Saved {new_filename} to {save_path}")

    return os.path.join(save_path, new_filename)  # Return path of processed file

In [ ]:
# Loop through each subfolder (rcp26, rcp45, rcp85)
for subfolder in subfolders:
    print(subfolder)
    folder_path = os.path.join(nc_files, subfolder)
    save_subfolder = os.path.join(general_path, 'tempDaysAbove', subfolder)

    # Create the destination subfolder if it doesn't exist
    os.makedirs(save_subfolder, exist_ok=True)

    # Choose the time ranges based on the subfolder
    if subfolder == 'historical':
        time_ranges = historical_time_range
    else:
        time_ranges = rcp_time_ranges

    # Initialize a dictionary to store processed files per threshold and time range
    processed_files_by_threshold = {threshold: [] for threshold in thresholds}

    # Loop through each NetCDF file in the subfolder
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)

        # Check if it's a NetCDF file (usually ends with .nc)
        if file.endswith('.nc'):
            # Loop through the temperature thresholds
            for threshold in thresholds:
                # Loop through the defined time ranges
                for start_year, end_year in time_ranges:
                    print(f"Processing threshold {threshold} for time range {start_year}-{end_year}")
                    
                    # Process and save the file with the new name for each time range
                    processed_file_path = process_file(file_path, threshold, save_subfolder, start_year, end_year)

print("Processing complete!")


## Average the Cordex models

In [ ]:
# Define base directory where the subfolders are located
base_dir = "/work/cmcc/dg07124/climax/indicators/cordex/tempDaysAbove"

# Define the subfolders for each scenario
subfolders = ['historical', 'rcp26', 'rcp45', 'rcp85']

# Dictionary to store the file paths grouped by subfolder, threshold, and time period
all_file_groups = {}

# Create a new folder for averaged models
averaged_results_dir = os.path.join(base_dir, 'avg_models')
os.makedirs(averaged_results_dir, exist_ok=True)

In [ ]:
# Loop over each subfolder separately
for subfolder in subfolders:
    folder_path = os.path.join(base_dir, subfolder)
    
    # Initialize a dictionary for each subfolder to store grouped files
    file_groups = defaultdict(list)
    
    # Loop over all files in the current subfolder
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".nc"):
            # Parse the threshold and time period from the filename
            parts = file_name.split("_")
            threshold = parts[-2]  # e.g., 'above45'
            time_period = parts[-1].replace(".nc", "")  # e.g., '2021-2050'
            
            # Create the group key based on threshold and time period
            group_key = (threshold, time_period)
            
            # Save the file path under its group
            file_path = os.path.join(folder_path, file_name)
            file_groups[group_key].append(file_path)
    
    # Save the grouped files for the current subfolder
    all_file_groups[subfolder] = file_groups

# Show the grouped file paths for verification
for subfolder, file_groups in all_file_groups.items():
    print(f"\nSubfolder: {subfolder}")
    for group_key, file_paths in file_groups.items():
        threshold, time_period = group_key
        print(f"Group: Threshold = {threshold}, Time Period = {time_period}")
        for file_path in file_paths:
            print(f"    - {file_path}")

# After verification, proceed with averaging the files for each group in each subfolder
for subfolder, file_groups in all_file_groups.items():
    for group_key, file_paths in file_groups.items():
        print("---------------------------------------")
        # print(group_key)
        # print(file_paths)
        
        # Unpack the group key (threshold and time_period)
        threshold, time_period = group_key  # Correctly use time_period from the group        
        
        # Load all datasets in the group
        datasets = [xr.open_dataset(fp) for fp in file_paths]
        # print(datasets)
        # Initialize a variable to hold the sum and count of valid datasets
        summed = None
        count = 0
        invalid_files = []  # To store the filenames with NaN values only

        for ds, fp in zip(datasets, file_paths):
            # Select the 'tx_days_above' variable and check for non-NaN values
            valid_data = ds['tasmax'].notnull()

            # Print min and max value for the current dataset
            min_value = ds['tasmax'].min().item()
            max_value = ds['tasmax'].max().item()
            print(f"File: {fp}")
            print(f"Minimum value: {min_value}")
            print(f"Maximum value: {max_value}")


            if valid_data.any():  # If there's at least one valid value
                if summed is None:
                    summed = ds['tasmax'].copy()  # Initialize summed with the first valid dataset
                else:
                    summed += ds['tasmax']  # Add to the sum
                count += 1  # Increment the count of valid datasets
            else:
                # If no valid data, add the file path to the invalid_files list
                invalid_files.append(fp)

        print(f"Number of valid datasets {count}")

        # Print filenames that are fully NaN
        if invalid_files:
            print("Files with NaN values:")
            for invalid_file in invalid_files:
                print(f"  - {invalid_file}")

        # Compute the average across the datasets only if count > 0
        if count > 0:
            averaged = summed / count
            
            # Define the subfolder for saving the averaged models for the current group
            subfolder_avg_dir = os.path.join(averaged_results_dir, subfolder)
            os.makedirs(subfolder_avg_dir, exist_ok=True)  # Create subfolder if it doesn't exist

            # Define output file path using the subfolder, threshold, and time period
            output_filename = f"{subfolder}_avg_{threshold}_{time_period}.nc"
            output_path = os.path.join(subfolder_avg_dir, output_filename)

            # Convert back to a dataset and save to a NetCDF file
            averaged_ds = averaged.to_dataset(name='tasmax')

            # Get the minimum and maximum values of the averaged dataset
            min_value_avg = averaged_ds['tasmax'].min().item()
            max_value_avg = averaged_ds['tasmax'].max().item()

            # Print the results for the averaged dataset
            print(f"Averaged Dataset: Minimum value: {min_value_avg}")
            print(f"Averaged Dataset: Maximum value: {max_value_avg}")
            averaged_ds.assign_coords({'lon' : ds.lon, 'lat':ds.lat})
            averaged_ds.to_netcdf(output_path)  # Save the averaged dataset
            print(f"Averaged data saved to: {output_path}")
        else:
            print("No valid datasets found for this group, skipping averaging.")



## Subtraction of the future scenarios from the historical datasets

In [ ]:
# Define the base directory containing the models
base_dir = "/work/cmcc/dg07124/climax/indicators/cordex/tempDaysAbove"
scenario_dir = os.path.join(base_dir, 'avg_models')
historical_dir = os.path.join(scenario_dir, 'historical')
output_folder = os.path.join(base_dir, 'subtracted_rcps') # output
os.makedirs(output_folder, exist_ok=True)

# Define the subfolders
scenario_subfolders = ['rcp26', 'rcp45', 'rcp85']

In [ ]:
# Loop over each scenario subfolder (rcp26, rcp45, rcp85)
for scenario in scenario_subfolders:
    scenario_folder = os.path.join(scenario_dir, scenario)

    # Loop through each file in the scenario subfolder
    for scenario_file in os.listdir(scenario_folder):
        if scenario_file.endswith(".nc"):
            # Extract the threshold from the scenario filename (e.g., 'above35' from 'rcp26_avg_above35_2021-2050.nc')
            scenario_parts = scenario_file.split("_")
            scenario_threshold = scenario_parts[-2]  # e.g., 'above35'

            # Find the corresponding historical file with the same threshold
            matching_historical_file = None
            for hist_file in os.listdir(historical_dir):
                if hist_file.endswith(".nc"):
                    hist_parts = hist_file.split("_")
                    historical_threshold = hist_parts[-2]  # e.g., 'above35'
                    
                    # Check if the thresholds match
                    if scenario_threshold == historical_threshold:
                        matching_historical_file = hist_file
                        break  # Exit loop once a matching file is found
            
            # If a matching historical file is found, proceed with subtraction
            if matching_historical_file:
                historical_file_path = os.path.join(historical_dir, matching_historical_file)

                print(f"Scenario file: {scenario_file}")
                print(f"Historical file: {matching_historical_file}")

                # Load both scenario and historical datasets
                scenario_ds = xr.open_dataset(os.path.join(scenario_folder, scenario_file))
                historical_ds = xr.open_dataset(historical_file_path)

                # Ensure both datasets contain the same variable ('tx_days_above') and then subtract
                if 'tasmax' in scenario_ds and 'tasmax' in historical_ds:
                    # Subtract historical from scenario
                    diff = scenario_ds['tasmax'] - historical_ds['tasmax']

                    # Save the difference to a new NetCDF file
                    scenario_time_period = scenario_parts[-1].replace(".nc", "")  # Extract the time period
                    diff_filename = f"diff_{scenario}_{scenario_threshold}_{scenario_time_period}.nc"
                    diff_filepath = os.path.join(output_folder, diff_filename)
                    diff = diff.assign_coords({'lon' : historical_ds.lon, 'lat': historical_ds.lat})
                    
                    # Save dataset to a local directory for future access
                    diff.to_dataset(name='tasmax').to_netcdf(diff_filepath)

                    print(f"Difference saved to: {diff_filepath}")
                else:
                    print(f"Variable 'tasmax' not found in one of the datasets.")
            else:
                print(f"Corresponding historical file not found for threshold: {scenario_threshold} in {scenario_file}")            


## Contributors
- Giuseppe Giugliano (giuseppe.giugliano@cmcc.it)
- Carmela de Vivo (carmela.devivo@cmcc.it)
- Daniela Quintero (daniela.quintero@cmcc.it)